In [ ]:
import pandas as pd

from neuralforecast.core import NeuralForecast
from neuralforecast.models.mixmodel_rescale_deco import MIXModel
from neuralforecast.losses.pytorch import MAE, MSE
from neuralforecast.models import Autoformer, FEDformer, Informer, PatchTST, NBEATS, NHITS, MLP
from datasetsforecast.long_horizon2 import LongHorizon2, LongHorizon2Info

input_size = 96
horizon = 192
dataset = "ETTh1"
epochs = 300
loss=MAE()
n_heads = 4
hidden_size = 64
batch_size = 32
learning_rate = 1e-4

# read data：
Y_df = LongHorizon2.load(directory="./data/", group=dataset)
Y_df["ds"] = pd.to_datetime(Y_df["ds"])
n_time = LongHorizon2Info[dataset].n_time
freq = LongHorizon2Info[dataset].freq
val_size = LongHorizon2Info[dataset].val_size
test_size = LongHorizon2Info[dataset].test_size
Y_df.drop(columns=["index"], inplace=True)

informer = Informer(
  h=horizon,
  input_size=input_size,
  loss = loss,
  max_steps = epochs,
  n_head = n_heads,
  hidden_size = hidden_size,
  batch_size = batch_size,
  learning_rate = learning_rate,
)

autoformer = Autoformer(
  h=horizon,
  input_size=input_size,
  loss = loss,
  max_steps = epochs,
  n_head = n_heads,
  hidden_size = hidden_size,
  batch_size = batch_size,
  learning_rate = learning_rate,
)

fedformer = FEDformer(
  h=horizon,
  input_size=input_size,
  loss = loss,
  max_steps = epochs,
  n_head = 8,
  hidden_size = hidden_size,
  batch_size = batch_size,
  learning_rate = learning_rate,
)

drnn = MLP(
  h=horizon,
  input_size=input_size,
  loss = loss,
  max_steps = epochs,
  batch_size = batch_size,
  learning_rate = learning_rate,
)

patchtst = PatchTST(
  h=horizon,
  input_size=input_size,
  loss = loss,
  max_steps = epochs,
  n_heads = n_heads,
  hidden_size = hidden_size,
  batch_size = batch_size,
  learning_rate = learning_rate,
)

nbeats = NBEATS(
  h=horizon,
  input_size=512,
  loss = loss,
  max_steps = epochs,
)

nhits = NHITS(
  h=horizon,
  input_size=512,
  loss = loss,
  max_steps = epochs,
)

models = [drnn, informer, autoformer, fedformer, patchtst, nbeats, nhits]

for model in models:
  # fit & test model
  nf = NeuralForecast(models=[model], freq=freq)
  Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size,step_size=1, n_windows=None)
  Y_hat_df.to_csv(f'./data/{dataset}_{horizon}_{nf.models[0]}.csv', index=False)

